In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay, roc_curve,auc, precision_recall_curve, average_precision_score, f1_score, auc 
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from imblearn.metrics import geometric_mean_score
import json
import seaborn as sns
import warnings
import pickle
import shap
warnings.filterwarnings("ignore")
sns.set_theme()

from tqdm import tqdm

In [25]:
for i in range(1,30+1):
    
    X_train = pd.read_csv(f'/camin1/chlee/jupyter/ML project/[24-12-13]/Data/X_train/X_train_{i}.csv')
    X_test = pd.read_csv(f'/camin1/chlee/jupyter/ML project/[24-12-13]/Data/X_test/X_test_{i}.csv')    
    y_train = pd.read_csv(f'/camin1/chlee/jupyter/ML project/[24-12-13]/Data/y_train/y_train_{i}.csv')
    y_test = pd.read_csv(f'/camin1/chlee/jupyter/ML project/[24-12-13]/Data/y_test/y_test_{i}.csv')

    globals()[f'X_train_{i}'] = X_train.drop(X_train.columns[0], axis=1)
    globals()[f'X_test_{i}'] = X_test.drop(X_test.columns[0], axis=1)
    globals()[f'y_train_{i}'] = y_train.drop(y_train.columns[0], axis=1)
    globals()[f'y_test_{i}'] = y_test.drop(y_test.columns[0], axis=1)

# Subgroup analysis - highrisk group

In [26]:
for i in range(1,30+1):
    
    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')
    
    X_train = X_train[X_train['FAMILY_HISTORY']==1][['AGE','MP_OXC','FHXNG','BRCACC_1','BRCACC_2','BMIG']]
    X_test = X_test[X_test['FAMILY_HISTORY']==1][['AGE','MP_OXC','FHXNG','BRCACC_1','BRCACC_2','BMIG']]

    y_train = y_train.iloc[X_train.index]
    y_test = y_test.iloc[X_test.index]
    

    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)

    globals()[f'X_train_{i}'] = X_train
    globals()[f'X_test_{i}'] = X_test
    
    y_train.reset_index(drop=True)
    y_test.reset_index(drop=True)
    
    globals()[f'y_train_{i}'] = y_train
    globals()[f'y_test_{i}'] = y_test

### Modeling

#### Logistic Regression

In [27]:
def LR_cv(search_space):
    model = LogisticRegression(solver = search_space['solver'],
                               max_iter = search_space['max_iter'],
                               random_state=42)
    
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies = []
    
    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        age_mean = np.mean(X_train_fold['AGE'])
        age_std = np.std(X_train_fold['AGE'])
        
        X_train_fold['AGE'] = (X_train_fold['AGE'] - age_mean) / age_std
        X_val_fold['AGE'] = (X_val_fold['AGE'] - age_mean) / age_std
        
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_probs = model.predict_proba(X_val_fold)[:,1]
        y_pred = model.predict(X_val_fold)
        
        accuracy = accuracy_score(y_val_fold, y_pred)
        accuracies.append(accuracy)
        
    return {'loss': -np.mean(accuracies), 'status': STATUS_OK}

In [28]:
search_space = {
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
    'max_iter': hp.choice('max_iter', range(100,1000))
}

In [ ]:
for i in range(1,30+1):

    print(f'{i}th loop')

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')
    
    trials = Trials()
    
    LR_ho = fmin(
        fn = LR_cv,
        space = search_space,
        algo = tpe.suggest,
        max_evals = 30,
        trials = trials
    )
    globals()[f'LR_ho_{i}'] = LR_ho
    LR_ho['max_iter'] = int(LR_ho['max_iter'])
    LR_ho['solver'] = int(LR_ho['solver'])

    with open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/hyperparameters/LR/LR_ho_{i}.json', 'w') as f:
        data = {key: value for key, value in LR_ho.items()}
        json.dump(data, f)
    print(LR_ho)
            
    solver_option = ['newton-cg', 'lbfgs', 'liblinear'][LR_ho['solver']]
    
    fit_LR = LogisticRegression(tol = 1e-4,
                                solver = solver_option,
                                max_iter = LR_ho['max_iter'],
                                random_state=42)
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])    
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_LR.fit(X_train, y_train)
    
    y_pred = fit_LR.predict(X_test)
    y_pred_probs = fit_LR.predict_proba(X_test)[:,1]
    
    print('>>>> ROC_AUC score after hyperparameter tuning:' f"{roc_auc_score(y_test, y_pred_probs):.3f}")
    print('>>>> accuracy after hyperparameter tuning:', f"{accuracy_score(y_test, y_pred):.3f}")

#### RF

In [30]:
def rf_cv(search_space):
    model = RandomForestClassifier(max_depth = int(search_space['max_depth']),
                                   min_samples_leaf = search_space['min_samples_leaf'],
                                   n_estimators = 100,
                                   random_state=42)
   
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies = []

    
    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        age_mean = np.mean(X_train_fold['AGE'])
        age_std = np.std(X_train_fold['AGE'])
        
        X_train_fold['AGE'] = (X_train_fold['AGE'] - age_mean) / age_std
        X_val_fold['AGE'] = (X_val_fold['AGE'] - age_mean) / age_std
        
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_probs = model.predict_proba(X_val_fold)[:,1]
        y_pred = model.predict(X_val_fold)
        
        accuracy = accuracy_score(y_val_fold, y_pred)
        accuracies.append(accuracy)
        
    return {'loss': -np.mean(accuracies), 'status': STATUS_OK}

In [31]:
search_space = {
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
    }

In [ ]:
for i in range(1,30+1):

    print(f'{i}th loop')

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')
    
    trials = Trials()
    
    rf_ho = fmin(
        fn = rf_cv,
        space = search_space,
        algo = tpe.suggest,
        max_evals = 30,
        trials = trials
    )
    globals()[f'rf_ho_{i}'] = rf_ho

    with open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/hyperparameters/RF/rf_ho_{i}.json', 'w') as f:
        data = {key: value for key, value in rf_ho.items()}
        json.dump(data, f)
    print(rf_ho)
    
    fit_rf = RandomForestClassifier(max_depth=int(rf_ho['max_depth']),
                                    min_samples_leaf = rf_ho['min_samples_leaf'],
                                    n_estimators = 100,
                                    random_state=42)
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])    
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_rf.fit(X_train, y_train)
    
    y_pred = fit_rf.predict(X_test)
    y_pred_probs = fit_rf.predict_proba(X_test)[:,1]
    
    print('>>>> ROC_AUC score after hyperparameter tuning:' f"{roc_auc_score(y_test, y_pred_probs):.3f}")
    print('>>>> accuracy after hyperparameter tuning:', f"{accuracy_score(y_test, y_pred):.3f}")

#### XGB

In [ ]:
def XGB_cv(search_space):
    model = xgb.XGBClassifier(max_depth = int(search_space['max_depth']),
                              learning_rate = search_space['learning_rate'],
                              n_estimators = 100,
                              gamma = search_space['gamma'],
                              min_child_weight = search_space['min_child_weight'],
                              max_delta_step = search_space['max_delta_step'],
                              subsample = search_space['subsample'],
                              colsample_bytree = search_space['colsample_bytree'],
                              scale_pos_weight = search_space['scale_pos_weight'],
                              seed=42)
        
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies = []
    
    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        age_mean = np.mean(X_train_fold['AGE'])
        age_std = np.std(X_train_fold['AGE'])
        
        X_train_fold['AGE'] = (X_train_fold['AGE'] - age_mean) / age_std
        X_val_fold['AGE'] = (X_val_fold['AGE'] - age_mean) / age_std
        
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_probs = model.predict_proba(X_val_fold)[:,1]
        y_pred = model.predict(X_val_fold)
        
        accuracy = accuracy_score(y_val_fold, y_pred)
        accuracies.append(accuracy)
        
    return {'loss': -np.mean(accuracies), 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': hp.quniform('max_depth', 1, 7, 1),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
    'gamma': hp.quniform('gamma', 0, 1, 0.1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 4, 1),
    'max_delta_step': hp.quniform('max_delta_step', 5, 9, 1),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.1),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.1),
    'scale_pos_weight': hp.quniform('scale_pos_weight', 2, 6, 1)
}

In [ ]:
for i in range(1,30+1):

    print(f'{i}th loop')

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')
    
    trials = Trials()
    
    xgb_ho = fmin(
        fn = XGB_cv,
        space = search_space,
        algo = tpe.suggest,
        max_evals = 30,
        trials = trials
    )
    globals()[f'xgb_ho_{i}'] = xgb_ho

    with open(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/hyperparameters/XGB/xgb_ho_{i}.json', 'w') as f:
        data = {key: value for key, value in xgb_ho.items()}
        json.dump(data, f)
    print(xgb_ho)
            
    fit_xgb = xgb.XGBClassifier(max_depth= int(xgb_ho['max_depth']),
                             learning_rate=xgb_ho['learning_rate'],
                             n_estimators=100,
                             gamma= xgb_ho['gamma'],
                             min_child_weight=xgb_ho['min_child_weight'],
                             max_delta_step=xgb_ho['max_delta_step'],
                             subsample=xgb_ho['subsample'],
                             colsample_bytree=xgb_ho['colsample_bytree'],
                             scale_pos_weight=xgb_ho['scale_pos_weight'])
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])    
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_xgb.fit(X_train, y_train)
    
    y_pred = fit_xgb.predict(X_test)
    y_pred_probs = fit_xgb.predict_proba(X_test)[:,1]
    
    print('>>>> ROC_AUC score after hyperparameter tuning:' f"{roc_auc_score(y_test, y_pred_probs):.3f}")
    print('>>>> accuracy after hyperparameter tuning:', f"{accuracy_score(y_test, y_pred):.3f}")

#### Catboost

In [36]:
def catboost_cv(search_space):
    model = CatBoostClassifier(max_depth = int(search_space['max_depth']),
                               learning_rate = search_space['learning_rate'],
                               n_estimators = 100,
                               min_child_samples = int(search_space['min_child_samples']),
                               subsample = search_space['subsample'],
                               colsample_bylevel = search_space['colsample_bylevel'],
                               scale_pos_weight = search_space['scale_pos_weight'],
                               random_seed=42,
                               logging_level = "Silent")
        
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    accuracies = []
    
    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        age_mean = np.mean(X_train_fold['AGE'])
        age_std = np.std(X_train_fold['AGE'])
        
        X_train_fold['AGE'] = (X_train_fold['AGE'] - age_mean) / age_std
        X_val_fold['AGE'] = (X_val_fold['AGE'] - age_mean) / age_std
        
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_probs = model.predict_proba(X_val_fold)[:,1]
        y_pred = model.predict(X_val_fold)
        
        accuracy = accuracy_score(y_val_fold, y_pred)
        accuracies.append(accuracy)
        
    return {'loss': -np.mean(accuracies), 'status': STATUS_OK}

In [37]:
search_space = {
    'max_depth': hp.quniform('max_depth', 1, 7, 1),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
    'min_child_samples': hp.quniform('min_child_samples', 10, 40, 10),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.1),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.4, 1, 0.1),
    'scale_pos_weight': hp.quniform('scale_pos_weight', 2, 6, 1)
}

In [ ]:
for i in range(1,30+1):

    print(f'{i}th loop')

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')
    
    trials = Trials()
    
    catboost_ho = fmin(
        fn = catboost_cv,
        space = search_space,
        algo = tpe.suggest,
        max_evals = 30,
        trials = trials
    )
    globals()[f'catboost_ho_{i}'] = catboost_ho

    with open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/hyperparameters/Catboost/catboost_ho_{i}.json', 'w') as f:
        data = {key: value for key, value in catboost_ho.items()}
        json.dump(data, f)
    print(catboost_ho)
            
    fit_catboost = CatBoostClassifier(max_depth= int(catboost_ho['max_depth']),
                                      learning_rate=catboost_ho['learning_rate'],
                                      n_estimators=100,
                                      min_child_samples = int(catboost_ho['min_child_samples']),
                                      subsample=catboost_ho['subsample'],
                                      colsample_bylevel=catboost_ho['colsample_bylevel'],
                                      scale_pos_weight=catboost_ho['scale_pos_weight'],
                                      logging_level="Silent")
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])    
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_catboost.fit(X_train, y_train)
    
    y_pred = fit_catboost.predict(X_test)
    y_pred_probs = fit_catboost.predict_proba(X_test)[:,1]
    
    print('>>>> ROC_AUC score after hyperparameter tuning:' f"{roc_auc_score(y_test, y_pred_probs):.3f}")
    print('>>>> accuracy after hyperparameter tuning:', f"{accuracy_score(y_test, y_pred):.3f}")

---

# import model pickle files

#### LR

In [39]:
for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    with open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/hyperparameters/LR/LR_ho_{i}.json', 'r') as data: #####
        LR_ho = json.load(data)

    solver_option = ['newton-cg', 'lbfgs', 'liblinear'][LR_ho['solver']]
    
    fit_LR = LogisticRegression(tol = 1e-4,
                                solver = solver_option,
                                max_iter = LR_ho['max_iter'],
                                random_state=42)    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_LR.fit(X_train, y_train)
    pickle.dump(fit_LR, open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/LR/fit_LR_{i}.pkl', 'wb')) #####

### Random Forest

In [40]:
for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    with open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/hyperparameters/RF/rf_ho_{i}.json', 'r') as data: #####
        rf_ho = json.load(data)
                
    fit_rf = RandomForestClassifier(max_depth=int(rf_ho['max_depth']),
                                    min_samples_leaf = rf_ho['min_samples_leaf'],
                                    n_estimators = 100,
                                    random_state=42)

    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
    
    fit_rf.fit(X_train, y_train)
    pickle.dump(fit_rf, open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/RF/fit_rf_{i}.pkl', 'wb')) #####

### XGB

In [ ]:
for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    with open(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/hyperparameters/XGB/xgb_ho_{i}.json', 'r') as data: #####
        xgb_ho = json.load(data)

    fit_xgb = xgb.XGBClassifier(max_depth= int(xgb_ho['max_depth']),
                             learning_rate=xgb_ho['learning_rate'],
                             n_estimators=100,
                             gamma= xgb_ho['gamma'],
                             min_child_weight=xgb_ho['min_child_weight'],
                             max_delta_step=xgb_ho['max_delta_step'],
                             subsample=xgb_ho['subsample'],
                             colsample_bytree=xgb_ho['colsample_bytree'],
                             scale_pos_weight=xgb_ho['scale_pos_weight'])
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_xgb.fit(X_train, y_train)
    pickle.dump(fit_xgb, open(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/model pkl save/XGB/fit_xgb_{i}.pkl', 'wb')) #####


### Catboost

In [41]:
for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    with open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/hyperparameters/Catboost/catboost_ho_{i}.json', 'r') as data: #####
        catboost_ho = json.load(data)

    fit_catboost = CatBoostClassifier(max_depth= int(catboost_ho['max_depth']),
                                      learning_rate=catboost_ho['learning_rate'],
                                      n_estimators=100,
                                      min_child_samples = int(catboost_ho['min_child_samples']),
                                      subsample=catboost_ho['subsample'],
                                      colsample_bylevel=catboost_ho['colsample_bylevel'],
                                      scale_pos_weight=catboost_ho['scale_pos_weight'],
                                      logging_level="Silent")
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
        
    fit_catboost.fit(X_train, y_train)
    pickle.dump(fit_catboost, open(f'/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/Catboost/fit_catboost_{i}.pkl', 'wb')) #####


---

# Model performance comparison

In [67]:
# import model
for i in range(1,30+1):
    globals()[f'fit_LR_{i}'] = pickle.load(open(f"/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/LR/fit_LR_{i}.pkl", 'rb'))
    globals()[f'fit_rf_{i}'] = pickle.load(open(f"/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/RF/fit_rf_{i}.pkl", 'rb'))
    globals()[f'fit_catboost_{i}'] = pickle.load(open(f"/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/Catboost/fit_catboost_{i}.pkl", 'rb'))
    globals()[f'fit_xgb_{i}'] = pickle.load(open(f"/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/XGB/fit_xgb_{i}.pkl", 'rb'))

In [68]:
import scipy

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

#### LR

In [69]:
Accuracy_LR = []
AUROC_LR = []
AP_LR = []
Sensitivity_LR = []
Specificity_LR = []
Youden_LR = []
f1_LR = []
gmean_LR = []


LR_fpr_ = []
LR_tpr_ = []
LR_roc_auc_ = []

LR_precision_ = []
LR_recall_ = []
LR_ap_ = []
    
for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    exec(f'model = fit_LR_{i}')
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
    
    y_pred_probs = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)
    
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1score = f1_score(y_test, y_pred, average='binary')
    youden_index = sensitivity + specificity - 1
    gmean = geometric_mean_score(y_test, y_pred)

    Accuracy_LR.append(accuracy_score(y_test, y_pred))
    AUROC_LR.append(roc_auc_score(y_test, y_pred_probs))
    AP_LR.append(average_precision_score(y_test, y_pred_probs))
    Sensitivity_LR.append(sensitivity)
    Specificity_LR.append(specificity)
    Youden_LR.append(youden_index)
    f1_LR.append(f1score)
    gmean_LR.append(gmean)
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
    roc_auc = roc_auc_score(y_test, y_pred_probs)

    LR_fpr_.append(fpr)
    LR_tpr_.append(tpr)
    LR_roc_auc_.append(roc_auc)

    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
    average_precision = average_precision_score(y_test, y_pred_probs)

    LR_precision_.append(precision)
    LR_recall_.append(recall)
    LR_ap_.append(average_precision)

In [ ]:
acc_m, acc_h = np.round(mean_confidence_interval(Accuracy_LR),3)
spe_m, spe_h = np.round(mean_confidence_interval(Specificity_LR),3)
sen_m, sen_h = np.round(mean_confidence_interval(Sensitivity_LR),3)
f1_m, f1_h = np.round(mean_confidence_interval(f1_LR),3)
auroc_m, auroc_h = np.round(mean_confidence_interval(AUROC_LR),3)
ap_m, ap_h = np.round(mean_confidence_interval(AP_LR),3)
gmean_m, gmean_h = np.round(mean_confidence_interval(gmean_LR),3)

# 신뢰구간
print(acc_m,'±', acc_h)
print(spe_m,'±', spe_h)
print(sen_m,'±', sen_h)
print(f1_m,'±', f1_h)
print(auroc_m,'±', auroc_h)
print(ap_m,'±', ap_h)
print(gmean_m,'±', gmean_h)

### Random Forest

In [71]:
Accuracy_RF = []
AUROC_RF = []
AP_RF = []
Sensitivity_RF = []
Specificity_RF = []
Youden_RF = []
f1_RF = []
gmean_RF = []

rf_fpr_ = []
rf_tpr_ = []
rf_roc_auc_ = []

rf_precision_ = []
rf_recall_ = []
rf_ap_ = []
    
for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    exec(f'model = fit_rf_{i}')
    
    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
    
    y_pred_probs = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)

        
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1score = f1_score(y_test, y_pred, average='binary')
    youden_index = sensitivity + specificity - 1
    gmean = geometric_mean_score(y_test, y_pred)

    Accuracy_RF.append(accuracy_score(y_test, y_pred))
    AUROC_RF.append(roc_auc_score(y_test, y_pred_probs))
    AP_RF.append(average_precision_score(y_test, y_pred_probs))
    Sensitivity_RF.append(sensitivity)
    Specificity_RF.append(specificity)
    Youden_RF.append(youden_index)
    f1_RF.append(f1score)
    gmean_RF.append(gmean)

    # for auroc curve and precision call
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
    roc_auc = roc_auc_score(y_test, y_pred_probs)

    rf_fpr_.append(fpr)
    rf_tpr_.append(tpr)
    rf_roc_auc_.append(roc_auc)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
    average_precision = average_precision_score(y_test, y_pred_probs)

    rf_precision_.append(precision)
    rf_recall_.append(recall)
    rf_ap_.append(average_precision)

In [ ]:
acc_m, acc_h = np.round(mean_confidence_interval(Accuracy_RF),3)
spe_m, spe_h = np.round(mean_confidence_interval(Specificity_RF),3)
sen_m, sen_h = np.round(mean_confidence_interval(Sensitivity_RF),3)
f1_m, f1_h = np.round(mean_confidence_interval(f1_RF),3)
auroc_m, auroc_h = np.round(mean_confidence_interval(AUROC_RF),3)
ap_m, ap_h = np.round(mean_confidence_interval(AP_RF),3)
gmean_m, gmean_h = np.round(mean_confidence_interval(gmean_RF),3)

# 신뢰구간
print(acc_m,'±', acc_h)
print(spe_m,'±', spe_h)
print(sen_m,'±', sen_h)
print(f1_m,'±', f1_h)
print(auroc_m,'±', auroc_h)
print(ap_m,'±', ap_h)
print(gmean_m,'±', gmean_h)

### XGB

In [74]:
Accuracy_XGB = []
AUROC_XGB = []
AP_XGB = []
Sensitivity_XGB = []
Specificity_XGB = []
Youden_XGB = []
f1_XGB = []
gmean_XGB = []
    
xgb_fpr_ = []
xgb_tpr_ = []
xgb_roc_auc_ = []
    
xgb_precision_ = []
xgb_recall_ = []
xgb_ap_ = []

for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    exec(f'model = fit_xgb_{i}')

    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])
    
    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
    
    y_pred_probs = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)

        
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1score = f1_score(y_test, y_pred, average='binary')
    youden_index = sensitivity + specificity - 1
    gmean = geometric_mean_score(y_test, y_pred)


    Accuracy_XGB.append(accuracy_score(y_test, y_pred))
    AUROC_XGB.append(roc_auc_score(y_test, y_pred_probs))
    AP_XGB.append(average_precision_score(y_test, y_pred_probs))
    Sensitivity_XGB.append(sensitivity)
    Specificity_XGB.append(specificity)
    Youden_XGB.append(youden_index)
    f1_XGB.append(f1score)
    gmean_XGB.append(gmean)


    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
    roc_auc = roc_auc_score(y_test, y_pred_probs)

    xgb_fpr_.append(fpr)
    xgb_tpr_.append(tpr)
    xgb_roc_auc_.append(roc_auc)

    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
    average_precision = average_precision_score(y_test, y_pred_probs)

    xgb_precision_.append(precision)
    xgb_recall_.append(recall)
    xgb_ap_.append(average_precision)

In [ ]:
acc_m, acc_h = np.round(mean_confidence_interval(Accuracy_XGB),3)
spe_m, spe_h = np.round(mean_confidence_interval(Specificity_XGB),3)
sen_m, sen_h = np.round(mean_confidence_interval(Sensitivity_XGB),3)
f1_m, f1_h = np.round(mean_confidence_interval(f1_XGB),3)
auroc_m, auroc_h = np.round(mean_confidence_interval(AUROC_XGB),3)
ap_m, ap_h = np.round(mean_confidence_interval(AP_XGB),3)
gmean_m, gmean_h = np.round(mean_confidence_interval(gmean_XGB),3)

# 신뢰구간
print(acc_m,'±', acc_h)
print(spe_m,'±', spe_h)
print(sen_m,'±', sen_h)
print(f1_m,'±', f1_h)
print(auroc_m,'±', auroc_h)
print(ap_m,'±', ap_h)
print(gmean_m,'±', gmean_h)

### catboost

In [76]:
Accuracy_CATBOOST = []
AUROC_CATBOOST = []
AP_CATBOOST = []
Sensitivity_CATBOOST = []
Specificity_CATBOOST = []
Youden_CATBOOST = []
f1_CATBOOST = []
gmean_CATBOOST = []

catboost_fpr_ = []
catboost_tpr_ = []
catboost_roc_auc_ = []

catboost_precision_ = []
catboost_recall_ = []
catboost_ap_ = []

for i in range(1,30+1):

    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')

    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')

    exec(f'model = fit_catboost_{i}')

    age_mean = np.mean(X_train['AGE'])
    age_std = np.std(X_train['AGE'])

    X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
    X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std

    y_pred_probs = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1score = f1_score(y_test, y_pred, average='binary')
    youden_index = sensitivity + specificity - 1
    gmean = geometric_mean_score(y_test, y_pred)

    Accuracy_CATBOOST.append(accuracy_score(y_test, y_pred))
    AUROC_CATBOOST.append(roc_auc_score(y_test, y_pred_probs))
    AP_CATBOOST.append(average_precision_score(y_test, y_pred_probs))
    Sensitivity_CATBOOST.append(sensitivity)
    Specificity_CATBOOST.append(specificity)
    Youden_CATBOOST.append(youden_index)
    f1_CATBOOST.append(f1score)
    gmean_CATBOOST.append(gmean)

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
    roc_auc = roc_auc_score(y_test, y_pred_probs)

    catboost_fpr_.append(fpr)
    catboost_tpr_.append(tpr)
    catboost_roc_auc_.append(roc_auc)

    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
    average_precision = average_precision_score(y_test, y_pred_probs)

    catboost_precision_.append(precision)
    catboost_recall_.append(recall)
    catboost_ap_.append(average_precision)


In [ ]:
acc_m, acc_h = np.round(mean_confidence_interval(Accuracy_CATBOOST),3)
spe_m, spe_h = np.round(mean_confidence_interval(Specificity_CATBOOST),3)
sen_m, sen_h = np.round(mean_confidence_interval(Sensitivity_CATBOOST),3)
f1_m, f1_h = np.round(mean_confidence_interval(f1_CATBOOST),3)
auroc_m, auroc_h = np.round(mean_confidence_interval(AUROC_CATBOOST),3)
ap_m, ap_h = np.round(mean_confidence_interval(AP_CATBOOST),3)
gmean_m, gmean_h = np.round(mean_confidence_interval(gmean_CATBOOST),3)

# 신뢰구간
print(acc_m,'±', acc_h)
print(spe_m,'±', spe_h)
print(sen_m,'±', sen_h)
print(f1_m,'±', f1_h)
print(auroc_m,'±', auroc_h)
print(ap_m,'±', ap_h)
print(gmean_m,'±', gmean_h)

### mean ROC curve

In [78]:
def mean_auroc(model_name):
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    for i in range(1,30+1):
        
        X_train = globals()[f'X_train_{i}']
        X_test = globals()[f'X_test_{i}']
        
        y_train = globals()[f'y_train_{i}']
        y_test = globals()[f'y_test_{i}']

        classifier = globals()[f'fit_{model_name}_{i}']

        age_mean = np.mean(X_train['AGE'])
        age_std = np.std(X_train['AGE'])
        
        X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
        X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std
                        
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X_test,
            y_test,
            ax=ax,)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    ax.plot(0)

    plt.show()
    
    return mean_fpr, mean_tpr, mean_auc, std_auc

In [ ]:
LR_mean_fpr, LR_mean_tpr, LR_mean_auc, LR_std_auc =  mean_auroc(model_name='LR')
RF_mean_fpr, RF_mean_tpr, RF_mean_auc, RF_std_auc =  mean_auroc(model_name='rf')
CATBOOST_mean_fpr, CATBOOST_mean_tpr, CATBOOST_mean_auc, CATBOOST_std_auc =  mean_auroc(model_name='catboost')
XGB_mean_fpr, XGB_mean_tpr, XGB_mean_auc, XGB_std_auc =  mean_auroc(model_name='xgb')

In [80]:
models = {'LR':{'mean_fpr': LR_mean_fpr, 'mean_tpr':LR_mean_tpr, 'mean_auc':LR_mean_auc, 'std_auc':LR_std_auc},
          'RF':{'mean_fpr': RF_mean_fpr, 'mean_tpr':RF_mean_tpr, 'mean_auc':RF_mean_auc, 'std_auc':RF_std_auc},
          'GBM':{'mean_fpr': XGB_mean_fpr, 'mean_tpr':XGB_mean_tpr, 'mean_auc':XGB_mean_auc, 'std_auc':XGB_std_auc},
          'Catboost':{'mean_fpr': CATBOOST_mean_fpr, 'mean_tpr':CATBOOST_mean_tpr, 'mean_auc':CATBOOST_mean_auc, 'std_auc':CATBOOST_std_auc}}

In [81]:
plt.figure(0)

for name, values in models.items():
    fpr = values['mean_fpr']
    tpr = values['mean_tpr']
    roc_auc = values['mean_auc']

    plt.plot(fpr, tpr, label='%s (AUC = %0.3f)'  % (name, roc_auc))
    plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.title('ROC curves')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')


plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/figure 1.tiff', format='tiff', dpi=300)  
plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/figure 1.pdf', format='pdf', dpi=300)  
plt.close()  

In [ ]:
plt.figure(0)

for name, values in models.items():
    fpr = values['mean_fpr']
    tpr = values['mean_tpr']
    roc_auc = values['mean_auc']

    plt.plot(fpr, tpr, label='%s (AUC = %0.3f)'  % (name, roc_auc))
    plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.title('ROC curves')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')


# plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/figure 1.tiff', format='tiff', dpi=300)  
# plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/figure 1.pdf', format='pdf', dpi=300)  
# plt.close()  

---

# Shap : risk factor analysis

### LR

In [42]:
LR_model = pickle.load(open('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/LR/fit_LR_1.pkl', 'rb'))

In [ ]:
X_train = X_train_1
X_test = X_test_1

y_train = y_train_1
y_test = y_test_1

age_mean = np.mean(X_train['AGE'])
age_std = np.std(X_train['AGE'])

X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std

LR_model.fit(X_train, y_train)

In [ ]:
predict_proba_function = lambda X: LR_model.fit(X_train, y_train).predict_proba(X)[:,1]

explainer = shap.KernelExplainer(predict_proba_function, shap.sample(X_train, 1000))
shap_values = explainer.shap_values(X_test)

In [47]:
shap_values_LR = shap_values

In [ ]:
shap_values_LR

In [49]:
# export as pickle file
with open('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_values_LR.pkl', 'wb') as file:
    pickle.dump(shap_values_LR, file)

In [ ]:
shap.summary_plot(shap_values_LR, X_test)

In [51]:
plt.figure(0)

shap.summary_plot(shap_values_LR, X_test, show=False)  

plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_summary_plot_LR.tiff', format='tiff', dpi=300)  
plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_summary_plot_LR.pdf', format='pdf', dpi=300) 

plt.close()

### RF

In [52]:
rf_model = pickle.load(open('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/RF/fit_rf_1.pkl', 'rb'))

In [ ]:
X_train = X_train_1
X_test = X_test_1

y_train = y_train_1
y_test = y_test_1

age_mean = np.mean(X_train['AGE'])
age_std = np.std(X_train['AGE'])

X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std

rf_model.fit(X_train, y_train)

In [ ]:
predict_proba_function = lambda X: rf_model.fit(X_train, y_train).predict_proba(X)[:,1]

explainer = shap.KernelExplainer(predict_proba_function, shap.sample(X_train, 1000))
shap_values = explainer.shap_values(X_test)

In [55]:
shap_values_rf = shap_values

In [ ]:
shap_values_rf

In [57]:
# export as pickle file
with open('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_values_rf.pkl', 'wb') as file:
    pickle.dump(shap_values_rf, file)

In [ ]:
shap.summary_plot(shap_values_rf, X_test)

In [59]:
plt.figure(0)

shap.summary_plot(shap_values_rf, X_test, show=False)  

plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_summary_plot_rf.tiff', format='tiff', dpi=300)  
plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_summary_plot_rf.pdf', format='pdf', dpi=300) 

plt.close()

### XGB

In [ ]:
xgb_model = pickle.load(open('C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/model pkl save/XGB/fit_xgb_1.pkl', 'rb'))

In [ ]:
X_train = X_train_1
X_test = X_test_1

y_train = y_train_1
y_test = y_test_1

age_mean = np.mean(X_train['AGE'])
age_std = np.std(X_train['AGE'])

X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std

xgb_model.fit(X_train, y_train)

In [ ]:
predict_proba_function = lambda X: xgb_model.fit(X_train, y_train).predict_proba(X)[:,1]

explainer = shap.KernelExplainer(predict_proba_function, shap.sample(X_train, 1000))
shap_values = explainer.shap_values(X_test)

In [ ]:
shap_values_xgb = shap_values

In [ ]:
shap_values_xgb

In [ ]:
# export as pickle file
with open('C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/shap/shap_values_xgb.pkl', 'wb') as file:
    pickle.dump(shap_values_xgb, file)

In [ ]:
shap.summary_plot(shap_values_xgb, X_test)

In [ ]:
plt.figure(0)

shap.summary_plot(shap_values_xgb, X_test, show=False)  

plt.savefig('C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/shap/shap_summary_plot_xgb.tiff', format='tiff', dpi=300)  
plt.savefig('C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/shap/shap_summary_plot_xgb.pdf', format='pdf', dpi=300) 

plt.close()

#### Catboost

In [60]:
catboost_model = pickle.load(open('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/model pkl save/Catboost/fit_catboost_1.pkl', 'rb'))

In [ ]:
X_train = X_train_1
X_test = X_test_1

y_train = y_train_1
y_test = y_test_1

age_mean = np.mean(X_train['AGE'])
age_std = np.std(X_train['AGE'])

X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std

catboost_model.fit(X_train, y_train)

In [ ]:
predict_proba_function = lambda X: catboost_model.fit(X_train, y_train).predict_proba(X)[:,1]

explainer = shap.KernelExplainer(predict_proba_function, shap.sample(X_train, 1000))
shap_values = explainer.shap_values(X_test)

In [63]:
shap_values_catboost = shap_values

In [64]:
# export as pickle file
with open('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_values_catboost.pkl', 'wb') as file:
    pickle.dump(shap_values_catboost, file)

In [ ]:
shap.summary_plot(shap_values_catboost, X_test)

In [66]:
plt.figure(0)

shap.summary_plot(shap_values_catboost, X_test, show=False)  

plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_summary_plot_catboost.tiff', format='tiff', dpi=300)  
plt.savefig('/camin1/chlee/jupyter/ML project/[24-12-13]/subgroup analysis/hr/shap/shap_summary_plot_catboost.pdf', format='pdf', dpi=300) 

plt.close()